# CENTRAL LIBRARY

In [2]:
#Please Install using pip3 install SQLALCHEMY
#This cell creates required tables
from sqlalchemy import *
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref, sessionmaker
from datetime import datetime, timedelta

engine = create_engine('sqlite:///lib.db', echo=False)#Set this to true if SQLALchemy logs are to be enabled
Base = declarative_base()
Session = sessionmaker(bind = engine)
session = Session()

class StudentModel(Base):
    #create table Students
    __tablename__='students'

    student_id=Column(Integer, primary_key=True)
    std_name=Column(Integer)
    std_surname=Column(String(80))
    borrowed_amt=Column(Integer)
    borrow=relationship('BorrowModel')
    
    #Initialisation Method
    
    def __init__(self, std_name,std_surname,borrowed_amt):
        self.std_name=std_name
        self.std_surname=std_surname
        self.borrowed_amt=borrowed_amt

    @classmethod
    def find_by_name(cls,std_name):
        return session.query(cls).filter(cls.std_name.like('%'+std_name+'%'))

    
    def save_to_db(self):
        db.session.add(self)
        db.session.commit()
        
    @classmethod
    def check_limit(cls,student_id):
        return cls.query.filter_by(student_id=student_id)
    

class BookModel(Base):
    #create Books table
    __tablename__="books"

    
    book_id=Column(Integer, primary_key=True)
    isbn=Column(String(10))
    bk_title=Column(String(80))
    bk_subject=Column(String(100))
    bk_author=Column(String(30))
    bk_quantity=Column(Integer)
    borrow=relationship('BorrowModel')

    #Initialisation Method
   
    def __init__(self, isbn, book_title,subject,authors,quantity):
        self.isbn=isbn
        self.bk_title=book_title
        self.bk_author=authors
        self.bk_subject=subject
        self.bk_quantity=quantity

    @classmethod
    def find_by_isbn(cls,isbn):
        return session.query(cls).filter(cls.isbn.like('%'+isbn+'%')).all()

    @classmethod
    def find_by_title(cls,bk_title):
        return session.query(cls).filter(cls.bk_title.like('%'+bk_title+'%')).all()
    
    @classmethod
    def find_by_subject(cls,bk_subject):
        return session.query(cls).filter(cls.bk_subject.like('%'+bk_subject+'%')).all()
    
    @classmethod
    def find_by_author(cls,bk_author):
        return session.query(cls).filter(cls.bk_author.like('%'+bk_author+'%')).all()
    
    def save_to_db(self):
        db.session.add(self)
        db.session.commit()
        
    
class BorrowModel(Base):
    #create Borrow Table
    __tablename__='borrows'

    borrow_id=Column(Integer, primary_key=True)
    student_id=Column(Integer,ForeignKey('students.student_id'))
    book_id=Column(String(80),ForeignKey('books.book_id'))
    borrowed_date=Column(Date)
    available_date=Column(Date)
    status=Column(String(30))
    borrow=relationship(StudentModel)
    book=relationship(BookModel)

    #Initialisation Method
    def __init__(self,student_id,book_id,status):
        self.student_id=student_id
        self.book_id=book_id
        self.status=status
        self.borrowed_date=datetime.now()
        self.available_date=datetime.now() + timedelta(days=7)
        
#This code creates all tables in lib.db        
Base.metadata.create_all(engine)

In [2]:
#I got a Book dataset from Kaggle https://www.kaggle.com/zygmunt/goodbooks-10k/data#books.csv, and laoding it in database.
#I'm loading only 300 entries to keep it simple
#RUN this cell only
import pandas as pd
import csv

items=[]
rows=pd.read_csv('books.csv')
df=pd.DataFrame(rows,columns=['id','book_id','isbn','authors','title','quantity'])
print(df)
for i in range(0,300):
    print(df.loc[i]['id'])
    std=BookModel(df.loc[i]['isbn'],df.loc[i]['title'],"NUll",df.loc[i]['authors'],int(df.loc[i]['quantity']))
    session.add(std)
session.commit()

        id   book_id        isbn  \
0        1   2767052   439023483   
1        2         3   439554934   
2        3     41865   316015849   
3        4      2657    61120081   
4        5      4671   743273567   
5        6  11870085   525478817   
6        7      5907   618260307   
7        8      5107   316769177   
8        9       960  1416524797   
9       10      1885   679783261   
10      11     77203  1594480001   
11      12  13335037    62024035   
12      13      5470   451524934   
13      14      7613   452284244   
14      15     48855   553296981   
15      16   2429135   307269752   
16      17   6148028   439023491   
17      18         5  043965548X   
18      19        34   618346252   
19      20   7260188   439023513   
20      21         2   439358078   
21      22  12232938   316166685   
22      23     15881   439064864   
23      24         6   439139600   
24      25    136251   545010225   
25      26       968   307277674   
26      27         1   43978

127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300


In [4]:
#Convert a datatype to JSON
import json

In [3]:
#inserts 3 users in table, student_id is set to auto increment
#The __init__ method is invoked
std=[StudentModel("Geethesh","Bhat",0),
StudentModel("Bob","Marley",0),
StudentModel("Joe","Harry",0)]

session.bulk_save_objects(std)
session.commit()

# This section makes sure everything is setup

In [15]:
#The Borrow table must be empty, entries will be populated when Students borrow books
#select * from students
rows=session.query(BorrowModel).all()
items=[]
for row in rows:
    item={
        "Student ID":row.student_id,
        "Book ID":row.book_id,
        "Borrow ID": row.borrow_id,
        "Borrowed Date":row.borrowed_date,
        "Available Date":row.available_date,
        "Status":row.status
        
    }
    items.append(item)
print(items)

[{'Student ID': 1, 'Book ID': '1', 'Borrow ID': 1, 'Borrowed Date': datetime.date(2020, 4, 16), 'Available Date': datetime.date(2020, 4, 23), 'Status': 'Issued'}]


In [7]:
#displays students table
#select * from students
rows=session.query(StudentModel).all()
items=[]
for row in rows:
    item={
        "id":row.student_id,
        "name":row.std_name,
        "surname":row.std_surname,
        "borrowed books": row.borrowed_amt
    }
    items.append(item)
json.dumps(items)

'[{"id": 1, "name": "Geethesh", "surname": "Bhat", "borrowed books": 0}, {"id": 2, "name": "Bob", "surname": "Marley", "borrowed books": 0}, {"id": 3, "name": "Joe", "surname": "Harry", "borrowed books": 0}]'

In [8]:
# Join on borrws and students table on borrows.student_id=students.student_id
#This should also be empty as there's no entry in borrow table to join
rows=session.query(BorrowModel).join(StudentModel)
items=[]
for row in rows:
#     for ent in row.StudentModel:
    item={
        "Student ID":row.student_id,
        "Book ID":row.book_id,
        "Borrow ID": row.borrow_id,
        "Borrowed Date":row.borrowed_date,
        "Available Date":row.available_date
        
    }
    items.append(item)
print(items)

[]


In [ ]:
# Join on borrws and books table on borrows.book_id=borrows.book_id
#This should also be empty as there's no entry in borrow table to join
rows=session.query(BorrowModel).join(BookModel).filter(BookModel.isbn=="4393483")
items=[]
for row in rows:
    item={
        "Available Date":row.available_date}
    items.append(item)
print(items)

# We have setup the environment
The use case mentioned in mail is implemented here

In [9]:
#We will be performing operations using the students we created earlier
rows=StudentModel.find_by_name("geeth")
items=[]
for row in rows:
#     for ent in row.StudentModel:
    item={
        "Student ID":row.student_id,
        "Book ID":row.std_name,
        "Borrow ID": row.std_surname,
        "Borrowed Date":row.borrowed_amt
        
    }
    items.append(item)
json.dumps(items)

'[{"Student ID": 1, "Book ID": "Geethesh", "Borrow ID": "Bhat", "Borrowed Date": 0}]'

# Search book by ISBN, Title, Author

In [7]:
#find books by title
#please feel free to change the title
rows=BookModel.find_by_title("mocking")
items=[]
#prints the selected rows
for row in rows:
    item={
        "Book_ID":row.book_id,
        "ISBN":row.isbn,
        "Book title":row.bk_title,
        "Book Author": row.bk_author,
        "Book Subject":row.bk_subject,
        "Quantity":row.bk_quantity
        
    }
    items.append(item)
json.dumps(items)

'[{"Book_ID": 4, "ISBN": "61120081", "Book title": "To Kill a Mockingbird", "Book Author": "Harper Lee", "Book Subject": "NUll", "Quantity": 2}, {"Book_ID": 20, "ISBN": "439023513", "Book title": "Mockingjay (The Hunger Games, #3)", "Book Author": "Suzanne Collins", "Book Subject": "NUll", "Quantity": 4}]'

In [8]:
 #find books by isbn
rows=BookModel.find_by_isbn("1416914285")
items=[]
#prints the selected rows
for row in rows:
    item={
        "Book_ID":row.book_id,
        "ISBN":row.isbn,
        "Book title":row.bk_title,
        "Book Author": row.bk_author,
        "Book Subject":row.bk_subject,
        "Quantity":row.bk_quantity
        
    }
    items.append(item)
json.dumps(items)

'[{"Book_ID": 51, "ISBN": "1416914285", "Book title": "City of Bones (The Mortal Instruments, #1)", "Book Author": "Cassandra Clare", "Book Subject": "NUll", "Quantity": 3}]'

In [9]:
 #find books by author
rows=BookModel.find_by_author("cass")
items=[]
#prints the selected rows
for row in rows:
    item={
        "Book_ID":row.book_id,
        "ISBN":row.isbn,
        "Book title":row.bk_title,
        "Book Author": row.bk_author,
        "Book Subject":row.bk_subject,
        "Quantity":row.bk_quantity
        
    }
    items.append(item)
json.dumps(items)

'[{"Book_ID": 51, "ISBN": "1416914285", "Book title": "City of Bones (The Mortal Instruments, #1)", "Book Author": "Cassandra Clare", "Book Subject": "NUll", "Quantity": 3}, {"Book_ID": 134, "ISBN": "1416914307", "Book title": "City of Glass (The Mortal Instruments, #3)", "Book Author": "Cassandra Clare", "Book Subject": "NUll", "Quantity": 4}, {"Book_ID": 153, "ISBN": "1416914293", "Book title": "City of Ashes (The Mortal Instruments, #2)", "Book Author": "Cassandra Clare", "Book Subject": "NUll", "Quantity": 3}, {"Book_ID": 183, "ISBN": "1416975861", "Book title": "Clockwork Angel (The Infernal Devices, #1)", "Book Author": "Cassandra Clare", "Book Subject": "NUll", "Quantity": 3}, {"Book_ID": 206, "ISBN": "62059939", "Book title": "The Selection (The Selection, #1)", "Book Author": "Kiera Cass", "Book Subject": "NUll", "Quantity": 4}]'

## Check users borrow quota i.e max 2, the function is called at the time of borrowing a book

In [4]:
def check_books(std_id):
    '''This cell checks students borrow quota. 
        First checks the quota, 
        if doesn't exceed then 
        it will be incremented every time a book
        is allocated'''
    rows=session.query(StudentModel).filter(StudentModel.student_id==std_id)
    items=[]
    for row in rows:
        item={
            "id":row.student_id,
            "name":row.std_name,
            "surname":row.std_surname,
            "borrowed books": row.borrowed_amt
        }
        items.append(item)
    if(row.borrowed_amt<2):
        return True
    else:
        return False
print(check_books(1))

True


In [28]:
'''This section allows user to borrow book
Note the entries in Borrows table automatically populates, also the next available date
The book can currently be borrowed using ISBN of the book'''
def check_availability(bookISBN,std_id):
    item=[]
    rw=session.query(BookModel).filter(and_(BookModel.isbn==bookISBN,BookModel.bk_quantity!=0)).one_or_none()
    if rw:
        item={
            "Book_ID":rw.book_id,
            "ISBN":rw.isbn,
            "Book title":rw.bk_title,
            "Book Author": rw.bk_author,
            "Book Subject":rw.bk_subject,
            "Quantity":rw.bk_quantity}
        if check_books(std_id) is True:
            return bookborrow(std_id,rw.book_id)
        else:
            return "You have already borrowed two books!"
        
    else:
        rows=session.query(BorrowModel).join(BookModel).filter(BookModel.isbn==bookISBN)
        items=[]
        for row in rows:
            item={"Available Date":row.available_date}
            items.append(item)
        if items:
            return "Book Not available, it will be available on {}".format(items[0]['Available Date'])

        return "Book Not available, we can't give a date as when this will be available"
    
def bookborrow(std_id,book_id):
    #updates students borrow_amt column by 1
    try:
        session.query(StudentModel).filter(StudentModel.student_id==std_id).update({StudentModel.borrowed_amt: StudentModel.borrowed_amt+1})
        session.commit()
    except:
        return "Error occured while updating borrow amount"
    
    #updates book availability
    try:
        session.query(BookModel).filter(BookModel.book_id==book_id).\
        update({BookModel.bk_quantity: BookModel.bk_quantity-1})
        session.commit()
    except:
        "Error updating book availability"

    try:
        borrow1=BorrowModel(std_id,book_id,"Issued")
        session.add(borrow1)
        session.commit()
    except:
        return "Error occured while upadting borrow table"
    return "Book successfully borrowed for 7 days"
    
bookISBN=str(input("Enter the ISBN of the book you want to borrow: "))
std_id=int(input("Enter your Student Identification number: "))
print(check_availability(bookISBN,std_id))

Enter the ISBN of the book you want to borrow439023483
Enter your Student Identification number1
Book successfully borrowed for 7 days


In [36]:
'''This section allows students to return the books, note the change in tables'''
def bookcheck(std_id):
    #This query joins BorrowModel and StudentModel on student_id.
    #Its used to filter student and check the status.
    rows=session.query(BorrowModel).join(StudentModel)\
    .filter(and_(StudentModel.borrowed_amt!=0,BorrowModel.student_id==std_id,\
                 not_(BorrowModel.status.contains('Returned'))))
    if rows:
        for row in rows:
            item={
        "Student ID":row.student_id,
        "Book ID":row.book_id,
        "Borrowed Date":row.borrowed_date,
        "Due Date":row.available_date,
        "Status":row.status}
            items.append(item)
        if len(items)!=0:
            return items
        return False
            
def bookreturn(std_id,bk_id):
    try:
        session.query(BorrowModel).filter(and_(BorrowModel.book_id==bk_id,BorrowModel.student_id==std_id)).\
        update({BorrowModel.status:"Returned"})

        session.query(BookModel).filter(BookModel.book_id==bk_id).\
        update({BookModel.bk_quantity: BookModel.bk_quantity+1})
        
        session.query(StudentModel).filter(StudentModel.student_id==std_id).\
        update({StudentModel.borrowed_amt: StudentModel.borrowed_amt-1})
        session.commit()
    except:
        return "Error Returning Book"
    
    return "Book Successfully returned"

        
items=[]
std_id=int(input("Please enter Student ID: "))

if bookcheck(std_id)!=0:
    print(items)
    bk_id=str(input("Enter the Book_ID from the list: "))
    for i in items:
        if i['Book ID']==bk_id:
            print(bookreturn(std_id,int(bk_id)))
        else:
            print("No such book is allocated")
    
else:
    print("Student ID doesn't exist OR you might not have borrowed that book")

please enter Student ID1
Student ID doesn't exist OR you might not have borrowed that book


In [7]:
session.query(BorrowModel).delete()
session.commit()

In [22]:
#I used this query to manipulate the book count
session.query(BookModel).filter(BookModel.book_id==1).\
update({BookModel.bk_quantity: BookModel.bk_quantity-1})
session.commit()

In [ ]:
#Useful statement when we are testing the code. Especially used to reset borrow limit by decrementing by 1
session.query(StudentModel).filter(StudentModel.student_id==1).\
update({StudentModel.borrowed_amt: StudentModel.borrowed_amt+1})
session.commit()

In [13]:
#to terminate any active session at the end
session.close()